In [3]:
import pandas as pd

In [5]:
dataset = pd.read_csv('./data/MultipleLinearRegressionData.csv')

In [7]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [9]:
X

array([[0.5, 3, 'Home'],
       [1.2, 4, 'Library'],
       [1.8, 2, 'Cafe'],
       [2.4, 0, 'Cafe'],
       [2.6, 2, 'Home'],
       [3.2, 0, 'Home'],
       [3.9, 0, 'Library'],
       [4.4, 0, 'Library'],
       [4.5, 5, 'Home'],
       [5.0, 1, 'Cafe'],
       [5.3, 2, 'Cafe'],
       [5.8, 0, 'Cafe'],
       [6.0, 3, 'Library'],
       [6.1, 1, 'Cafe'],
       [6.2, 1, 'Library'],
       [6.9, 4, 'Home'],
       [7.2, 2, 'Cafe'],
       [8.4, 1, 'Home'],
       [8.6, 1, 'Library'],
       [10.0, 0, 'Library']], dtype=object)

In [11]:
from sklearn.compose import ColumnTransformer # Column Transformer은 여러 transformer 을 column에 좀 더 쉽게 적용하도록 한 클래스 - 열 변환기
from sklearn.preprocessing import OneHotEncoder # 원핫 인코딩을 위한 클래스

In [12]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'), [2])], remainder='passthrough')
'''
 첫번째 - 'encoder' : 원핫 인코딩을 적용하기 위해 사용
 두번째 - OneHotEncoder : 첫번째 인코딩을 수행할 클래스(원핫인코더) 객체를 넣어준다,
 (drop='first'), [2] : 다중 공정성 문제를 해결하기 위해 첫번째 컬럼을 삭제하고 2개만 사용하겠다.  2 인덱스 컬럼에 적용하겠다.
 remainder='passthrough' : 원하는 코딩을 적용하지 않는 데이터들은 그냥 그대로 둔다.
'''

"\n 첫번째 - 'encoder' : 원핫 인코딩을 적용하기 위해 사용\n 두번째 - OneHotEncoder : 첫번째 인코딩을 수행할 클래스(원핫인코더) 객체를 넣어준다,\n (drop='first'), [2] : 다중 공정성 문제를 해결하기 위해 첫번째 컬럼을 삭제하고 2개만 사용하겠다.  2 인덱스 컬럼에 적용하겠다.\n remainder='passthrough' : 원하는 코딩을 적용하지 않는 데이터들은 그냥 그대로 둔다.\n"

In [13]:
# 원하는 값을 인코딩 후에 다시 X 에 입력한다.
X = ct.fit_transform(X)
X

# 1 0 : Home
# 0 1 : Library
# 0 0 : Cafe

array([[1.0, 0.0, 0.5, 3],
       [0.0, 1.0, 1.2, 4],
       [0.0, 0.0, 1.8, 2],
       [0.0, 0.0, 2.4, 0],
       [1.0, 0.0, 2.6, 2],
       [1.0, 0.0, 3.2, 0],
       [0.0, 1.0, 3.9, 0],
       [0.0, 1.0, 4.4, 0],
       [1.0, 0.0, 4.5, 5],
       [0.0, 0.0, 5.0, 1],
       [0.0, 0.0, 5.3, 2],
       [0.0, 0.0, 5.8, 0],
       [0.0, 1.0, 6.0, 3],
       [0.0, 0.0, 6.1, 1],
       [0.0, 1.0, 6.2, 1],
       [1.0, 0.0, 6.9, 4],
       [0.0, 0.0, 7.2, 2],
       [1.0, 0.0, 8.4, 1],
       [0.0, 1.0, 8.6, 1],
       [0.0, 1.0, 10.0, 0]], dtype=object)

### 데이터 세트 분리

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 20 %

### 학습 (다중 선형 회귀)

In [21]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

### 예측 값과 실제 값 비교 (테스트 세트)

In [24]:
y_pred = reg.predict(X_test)
y_pred # 예측 값

array([ 92.15457859,  10.23753043, 108.36245302,  38.14675204])

In [26]:
y_test # 실제 값

array([ 90,   8, 100,  38], dtype=int64)

In [28]:
reg.coef_ # 독립변수 (공부장소(집) , 공부장소(도서관), 공부시간, 결석)
# 마이너스면 나쁜 결과 즉 집에서 공부하는게 가장 나쁜 결과 카페는 0이므로
# 공부시간은 1시간씩 증가할때마다 10.4점씩 올라가고, 결석은 1번 할때마다
# 1.64점씩 나쁜 영향을 준다.

array([-5.82712824, -1.04450647, 10.40419528, -1.64200104])

In [30]:
reg.intercept_ # y 절편

5.365006706544747

### 모델 평가

In [33]:
reg.score(X_train, y_train) # 훈련 세트 - 96.2점

0.9623352565265527

In [35]:
reg.score(X_test, y_test) # 테스트 세트 - 98.59점

0.9859956178877445

### 다양한 평가 지표 (회귀 모델)

R2 는 1에 가까울수록, 나머지는 0에 가까울수록 좋음

In [39]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred) # 실제 값, 예측 값 # MAE

3.2253285188288023

In [41]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred) # MSE

19.900226981515015

In [53]:
from math import sqrt
sqrt(mean_squared_error(y_test, y_pred))

4.460967045553578

In [55]:
# mean_squared_error(y_test, y_pred, squared=False) # RMSE. squared=False -> 제곱하지 말라는 뜻 1.6 에서 제거됨.

In [57]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred) # R2 . 선형회귀 모델평가가 이 R2 스코어 로 계산 된다.

0.9859956178877445